In [1]:
# -*- coding:utf-8 -*-
import pandas as pd
import subprocess
import pydotplus
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from IPython.display import Image
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import pipeline

# 数据导入
xls_file = pd.ExcelFile('alliance.xls')
# print(xls_file.sheet_names)
df = xls_file.parse('sheet1')
df_target = df['result']

In [2]:
# 对范畴类变量进行编码
for column in ['uristem','uriquery','http_status']:
    dummies = pd.get_dummies(df[column])
    df[dummies.columns]=dummies

# 去掉已编码的原属性
df.drop(['domain','uristem','uriquery','http_status'], axis=1, inplace=True)
#df.ix[:,df.columns!=999]

In [3]:
# 拆分数据集为训练集和测试集
train, test = train_test_split(df, test_size = 0.2)   # doing split of training and testing
train_data, train_target= train.ix[:, train.columns != 'result'], train['result']
test_data, test_target = test.ix[:, test.columns != 'result'], test['result']
print(train_data.shape)
print(train_target.shape)
print(train_data)

(1244, 19)
(1244,)
     referer影响  客户端跳转  标题敏感字  标题新闻字  超链接数量  外链数量  敏感外链数量  引用图片数量 引用外部js  \
1434     False  False   True   True    538    94       3     125  False   
918      False  False   True   True    425    39       7      24   True   
854      False  False  False  False      0     0       0       0  False   
1220     False  False   True   True    526   125       0      25   True   
894      False  False   True   True    483   183       5      71   True   
148      False  False   True   True    753     6      19      81   True   
693      False  False  False  False     12     1       0       2   True   
1475     False  False   True   True    394    98       2      32  False   
1452     False  False  False   True    524   519       0      28   True   
912      False  False   True   True    532   132       1      40  False   
599      False  False  False   True    142    83       5      84  False   
346       True  False  False  False    129     4       0       8   True   
594   

In [4]:
# 套用分类器
clf = RandomForestClassifier() #tweak the parameters yourself
clf.fit(train_data,train_target) #if you're passing in a sparse matrix, apply .toarray() to X
print(clf.feature_importances_)
print(clf.transform(train_data))

[  3.14387566e-02   0.00000000e+00   3.15909779e-02   2.83590804e-02
   1.93877955e-01   1.77201381e-01   7.02581190e-02   1.88797491e-01
   2.62052275e-02   1.49286085e-04   8.78463863e-02   2.48375229e-02
   2.35148390e-03   9.06411776e-02   1.67351321e-02   1.72555358e-02
   5.70191641e-03   6.75257049e-03   0.00000000e+00]
[[ 538.   94.    3.  125.    0.    0.]
 [ 425.   39.    7.   24.    6.    1.]
 [   0.    0.    0.    0.    0.    0.]
 ..., 
 [ 387.    6.    1.   74.    0.    0.]
 [ 105.    5.    4.   26.    0.    1.]
 [   0.    0.    0.    0.    0.    0.]]


/home/v/anaconda3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


In [5]:
# 交叉验证
train_scores = cross_val_score(clf , test_data, test_target, cv=7,scoring='accuracy')
print(train_scores)
#test_scores = cross_val_score(clf, test_data, test_target, cv=5,scoring='accuracy')
#print(scores)

[ 0.8         0.77777778  0.82222222  0.88888889  0.77777778  0.84090909
  0.72093023]
